In [1]:
import cv2
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

In [2]:
camera = Camera.instance(width = 224, height = 224) # 224*224*3
image = widgets.Image(format = 'jpeg', width = 224, height = 224)
cv2_img = widgets.Image(format = 'jpeg', width = 224, height = 224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform = bgr8_to_jpeg) # camera의 value와 image widget의 value 동기화(실시간)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
display(cv2_img)

Image(value=b'', format='jpeg', height='224', width='224')

In [23]:
def preprocess(camera_value):
    global camera #전처리를 위하여 전역함수 camera 설정.
    image_value = camera.value # camera value의 형태는 numpy.ndarray
    height, width = image_value.shape[:2] # (224, 224, 3)에서 224*224만 추출
    img_hsv = cv2.cvtColor(image_value, cv2.COLOR_BGR2HSV) # HSV가 직관적으로 사용가능.
    lower_blue = (30, 70, 30)  # (색상, 채도, 명도)
    upper_blue = (90, 255, 180)
    img_Mask = cv2.inRange(img_hsv, lower_blue, upper_blue) # Mask 작업을 통하여 어떤 부분만 이미지를 남길것인가 결정.
    img_bitwise = cv2.bitwise_and(image_value, image_value, mask = img_Mask) # bit연산(and)을 통하여 masking
    tmp_img = cv2.imencode(".jpeg", img_bitwise)[1].tostring() # bitwise를 jpeg로 code
    return tmp_img

In [5]:
def update(change):
    global cv2_img
    x = change['new']
    x = preprocess(x)
    cv2_img.value = x
    
update({'new': camera.value}) # init update

In [6]:
camera.observe(update, names='value')